In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
# Read the CSV file
prior_data = pd.read_csv("./database/order_products__prior.csv")
train_data = pd.read_csv("./database/order_products__train.csv")
orders_data = pd.read_csv("./database/orders.csv")
products_data = pd.read_csv("./database/products.csv")

print(orders_data.head())

prior_data = prior_data.values
train_data = train_data.values
orders_data = orders_data.values
products_data = products_data.values

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  


Select Top 100 products

In [6]:
def reorder_Top_100_product_popular():
    product_prior_data = prior_data[:,1]
    product_train_data = train_data[:,1]
    reordered_prior_data = prior_data[:,3]
    reordered_train_data = prior_data[:,3]
    product = np.concatenate((product_train_data, product_prior_data))
    reorder = np.concatenate((reordered_prior_data, reordered_train_data))
    reorder_Matrix = np.zeros(len(products_data[:,0])+1)
    for i in range(0, len(product)):
        if reorder[i] == 1:
            reorder_Matrix[product[i]] += 1
    dic = {}
    for i in range(1, len(reorder_Matrix)):
      dic[i-1] = (int)(reorder_Matrix[i])

    sorted_dict = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    # print(sorted_dict)
    top_100_popular_productsID = np.zeros((100,1),dtype=int)
    for i in range(0, 100):
      top_100_popular_productsID[i] = sorted_dict[i][0]+1
    
    return top_100_popular_productsID

top_100_popular_productsID = reorder_Top_100_product_popular()

Merge prior.csv file and train.csv file, and delete add_to_cart_order column

In [7]:
def mergeTwoFiles(train_data, prior_data):
  m = len(train_data)
  n = len(prior_data)
  merged = np.zeros((m+n+1,3),dtype=int)
  p1 = 0
  p2 = 0
  while p1 < m or p2 < n:
    if p1 == m:
      merged[p1+p2+1][0] = prior_data[p2][0]
      merged[p1+p2+1][1] = prior_data[p2][1]
      merged[p1+p2+1][2] = prior_data[p2][3]
      p2 = p2 + 1
    elif p2 == n:
      merged[p1+p2+1][0] = prior_data[p1][0]
      merged[p1+p2+1][1] = prior_data[p1][1]
      merged[p1+p2+1][2] = prior_data[p1][3]
      p1 = p1 + 1
    elif train_data[p1][0] < prior_data[p2][0]:
      merged[p1+p2+1][0] = train_data[p1][0]
      merged[p1+p2+1][1] = train_data[p1][1]
      merged[p1+p2+1][2] = train_data[p1][3]
      p1 = p1 + 1
    else:
      merged[p1+p2+1][0] = prior_data[p2][0]
      merged[p1+p2+1][1] = prior_data[p2][1]
      merged[p1+p2+1][2] = prior_data[p2][3]
      p2 = p2 + 1

  return merged

merged = mergeTwoFiles(train_data, prior_data)

In [8]:
merged[0]

array([0, 0, 0])

Select Top 100 popular products from the whole order

In [9]:
def dataset_Top_100():
  # The whole dataset is 33,819,107
  data_length = 300000  #3,000,000 (3/33) of the whole dataset
  dataset_top_100 = np.zeros((len(merged)+1,3),dtype=int)
  counter = 1
  with tqdm(range(1, data_length)) as loop:
    for i in loop:
      for j in range(0, 100):
        if merged[i][1] == top_100_popular_productsID[j][0]:
          dataset_top_100[counter] = merged[i]
          counter += 1
          break
  return dataset_top_100

dataset = dataset_Top_100()

100%|██████████| 299999/299999 [00:11<00:00, 26173.79it/s]


In [10]:
def resize_dataset_pos():
  pos = 0
  for i in range(1, len(dataset)):
    if(dataset[i][0] == 0):
      pos = i
      break
  return pos

pos = resize_dataset_pos()

In [11]:
dataset = np.resize(dataset,(pos,3))

In [12]:
print("order_id	product_id	reordered")
for i in range(0,10):
  print(dataset[i])

print(len(orders_data))

order_id	product_id	reordered
[0 0 0]
[    1 49683     0]
[    1 13176     0]
[    1 47209     0]
[    1 22035     1]
[    2 28985     1]
[    2 17794     1]
[    3 24838     1]
[    3 21903     1]
[    3 46667     1]
3421083


In [13]:
def sort_orders_data_by_orders():
  dic = {}
  for i in range(0, len(orders_data)):
    dic[orders_data[i][0]] = orders_data[i]
  
  sorted_dict = sorted(dic.items(), key=lambda x: x[0])
  for i in range(0, len(orders_data)):
    orders_data[i] = sorted_dict[i][1]

  return orders_data

orders_data = sort_orders_data_by_orders()
  

In [14]:
print("order_id  user_id eval_set  order_number  order_dow  order_hour_of_day days_since_prior_order")
for i in range(0,10):
  print(orders_data[i])

print(len(orders_data))

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day days_since_prior_order
[1 112108 'train' 4 4 10 9.0]
[2 202279 'prior' 3 5 9 8.0]
[3 205970 'prior' 16 5 17 12.0]
[4 178520 'prior' 36 1 9 7.0]
[5 156122 'prior' 42 6 16 9.0]
[6 22352 'prior' 4 1 12 30.0]
[7 142903 'prior' 11 2 14 30.0]
[8 3107 'prior' 5 4 6 17.0]
[9 139016 'prior' 14 0 19 5.0]
[10 135442 'prior' 4 6 8 8.0]
3421083


In [15]:
def add_dow_and_hours():
  final_dataset = np.zeros((len(dataset),5),dtype=int)
  i = 1
  j = 0
  for i in range(1, len(dataset)):
    final_dataset[i][0] = dataset[i][0]
    final_dataset[i][1] = dataset[i][1]
    final_dataset[i][2] = orders_data[dataset[i][0]-1][4]
    final_dataset[i][3] = orders_data[dataset[i][0]-1][5]
    final_dataset[i][4] = dataset[i][2]

  return final_dataset

final_dataset = add_dow_and_hours()

In [16]:
print("order_id  product_id  order_dow  order_hour_of_day  reordered")
for i in range(0,15):
  print(final_dataset[i])

order_id  product_id  order_dow  order_hour_of_day  reordered
[0 0 0 0 0]
[    1 49683     4    10     0]
[    1 13176     4    10     0]
[    1 47209     4    10     0]
[    1 22035     4    10     1]
[    2 28985     5     9     1]
[    2 17794     5     9     1]
[    3 24838     5    17     1]
[    3 21903     5    17     1]
[    3 46667     5    17     1]
[    5 13176     6    16     1]
[    5 27966     6    16     1]
[    5 23909     6    16     1]
[    5 47209     6    16     0]
[    9 31506     0    19     1]


In [17]:
top_100_popular_productsID = np.sort(np.resize(top_100_popular_productsID,(100,)))

In [18]:
def new_product_table():
  dic = {}
  new_products = []
  j = 0
  for i in range(0, len(products_data)):
    if products_data[i][0] == top_100_popular_productsID[j]:
      new_products.append([top_100_popular_productsID[j],products_data[i][1]])
      j = j + 1
    if j == 100:
      break
  return new_products

new_products = new_product_table()

In [19]:
print(new_products[0])
print(new_products[-1])

[196, 'Soda']
[49683, 'Cucumber Kirby']


In [20]:
def rename_productID():
  for i in range(0, len(final_dataset)):
    for j in range(0, len(new_products)):
      if final_dataset[i][1] == new_products[j][0]:
        final_dataset[i][1] = j + 1
        break
  # change to 100 classifier
  for i in range(0, len(final_dataset)):
    final_dataset[i][1] = final_dataset[i][1]-1
    if i%10 == 0:
      final_dataset[i][4] = final_dataset[i][1]
    else:
      final_dataset[i][4] = final_dataset[i][4]+i%10
rename_productID()

In [21]:
for i in range(0,len(new_products)):
  new_products[i][0] = i+1

Save the final version as dataset.

order_id
product_id
order_dow
order_hour_of_day
reordered

In [22]:
np.savetxt('./database/dataset.csv', final_dataset, delimiter=',',fmt='%d')

In [23]:
name = ['productID','productName']
new_products_table = pd.DataFrame(columns=name,data=new_products)
new_products_table.to_csv('./database/new_porducts.csv')